# 実習14-2 画像分類モデルの保存と読み込み

今回は、画像分類APIの作成を実施する。ここでは、MobileNetを使った転移学習でモデルを作成し、猫と犬の画像を分類するプログラムを作成する。

**※ 基本的には、実習10-2の一部にkerasモデル保存を追加した内容です。**

**※ できればGPUを使用した方がいいと思います。**


基本的な流れは以下のようになる。
*   元になるモデルを作成し、事前トレーニング済みの重みを読み込む。
*   trainable = False を設定して、ベースモデルのすべての層を凍結する
*   出力したいものを考慮し、ベースモデルに層を追加して新しいモデルを作成する。
*   新しいデータセットで新しいモデルをトレーニングする。

## 1. データセットの準備
犬と猫の画像データセットを使用する。

### 1-1. zipファイルのダウンロードと展開
犬と猫の画像データセットをダウンロードして展開する。実習10-2のプログラムをコピーして実施してよい。

In [1]:
# 必要に応じてimport文を書く
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

In [2]:
#画像サイズ（実行しておく）：今回は元モデルの入力が224x224のため
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [3]:
# ダウンロード元URL
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

# get_fileを使用してzipをダウンロードし展開
# ダウンロード先（cache_subdir）は必要に応じて変更する
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True, cache_subdir='/content/datasets')

68606236/68606236 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### 1-2. ImageDataGeneratorによるデータセットの作成
ImageDataGeneratorのセットアップ部分をコピーして実施する。訓練用と検証用のものをそれぞれ作成すること。
**※画像データの拡張は、してもしなくてもOK**

In [4]:
# ImageDataGeneratorのセットアップ
from tensorflow.keras.preprocessing.image import ImageDataGenerator

次に、flow_from_directoryを使用し、画像のあるディレクトリ等を指定する。すると、データセットを自動的に作成する。訓練用と検証用のデータセットをそれぞれ作成すること。
**seedには学籍番号を指定すること。**

In [5]:
# 訓練用
train_img_gen = ImageDataGenerator(rescale=1./255)
train_data_it = train_img_gen.flow_from_directory(directory='/content/datasets/cats_and_dogs_filtered/train',
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary',
                                                           seed=2220042)

Found 2000 images belonging to 2 classes.


In [6]:
# 検証用
val_img_gen = ImageDataGenerator(rescale=1./255)
val_data_it = val_img_gen.flow_from_directory(directory='/content/datasets/cats_and_dogs_filtered/validation',
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary',
                                                           seed=2220042)

Found 1000 images belonging to 2 classes.


## 2. MobileNetを使用した転移学習

### 2-1. MobileNetを転移学習用に読み込み

MobileNetを転移学習用に読み込む場合は、この場合は、モデル作成時に引数としてinclude_top=Falseを指定する。

また、重みパラメータはImageNetで学習済みのものを読み込む。

**※ VGGを使用すると、モデル使用時にt2.microのEC2ではメモリ不足になる可能性が高いです。**
```
# MobileNetを転移学習のベースモデルとして読み込み
base_model = MobileNet(weights='imagenet',input_shape=(IMG_HEIGHT,IMG_WIDTH,3),include_top=False)

# base_modelのサマリを表示
base_model.summary()
```

In [7]:
# MobileNetを転移学習のベースモデルとして読み込み
base_model = MobileNet(weights='imagenet',input_shape=(IMG_HEIGHT,IMG_WIDTH,3),include_top=False)

# base_modelのサマリを表示
base_model.summary()

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_3 (Conv2D)                   │ (None, 56, 56, 128)         │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 3,206,976 (12.23 MB)

 Non-trainable params: 21,888 (85.50 KB)

多くのパラメータがTrainable（学習可能）になっているので、学習を不可にしておく。
```
# モデルの凍結
base_model.trainable = False
# summaryの確認
base_model.summary()
```

In [8]:
# モデルの凍結
base_model.trainable = False
# summaryの確認
base_model.summary()

Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_3 (Conv2D)                   │ (None, 56, 56, 128)         │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 3,228,864 (12.32 MB)

**パラメータがすべてTrainableからNon-Trainableになっていることを確認すること。**

### 2-2. 新しいモデルの作成
上で作成したベースモデルに層を追加して新しいモデルを作成する。今回は、入力の形状を指定し、後半にFlatten（1次元にする）、全結合層、出力層を追加してみよう。

（Functional APIでないと作れないかもしれない）

```
# 短く書くためにインポート
from keras.layers import Input, Flatten, Dense
from keras.models import Model

# 入力層を作成する

# ベースモデルに層を追加する。

# Modelの引数として入力（inputs）と出力（outputs）を指定してモデルを作成する

# summaryを表示して確認


```

In [9]:
# 短く書くためにインポート
from keras.layers import Input, Flatten, Dense
from keras.models import Model

# 入力層を作成する
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# ベースモデルに層を追加する。
x = base_model(inputs)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Modelの引数として入力（inputs）と出力（outputs）を指定してモデルを作成する
model = Model(inputs=inputs, outputs=predictions)

# summaryを表示して確認
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenet_1.00_224 (Functional)      │ (None, 7, 7, 1024)          │       3,228,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       3,211,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,440,257 (24.57 MB)

 Trainable params: 3,211,393 (12.25 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

### 2-3. 新しいデータで学習・評価
MobileNetを使用して作成したモデルを、学習させたいデータ（犬と猫）で学習させる。

```
# モデルのコンパイル
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fitで学習しつつ検証（とりあえずepochs=5で）
hist = model.fit(train_data_it,validation_data= val_data_it, epochs=5)
```

In [10]:
# モデルのコンパイル
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fitで学習しつつ検証（とりあえずepochs=5で）
hist = model.fit(train_data_it,validation_data= val_data_it, epochs=5)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 217ms/step - accuracy: 0.8531 - loss: 2.0963 - val_accuracy: 0.9810 - val_loss: 0.2275
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - accuracy: 0.9860 - loss: 0.2354 - val_accuracy: 0.9750 - val_loss: 0.4530
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - accuracy: 0.9932 - loss: 0.0569 - val_accuracy: 0.9750 - val_loss: 0.3558
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - accuracy: 0.9904 - loss: 0.1165 - val_accuracy: 0.9810 - val_loss: 0.3804
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step - accuracy: 0.9907 - loss: 0.1654 - val_accuracy: 0.9890 - val_loss: 0.1408


### 2-4. モデルの保存
モデルファイル名.saveを使用してモデルをファイルに保存する。

今回は、ファイル名を.iris学籍番号.kerasにしておく（例：catsdogs2220999.keras）
```
# モデルの保存
model.save('catsdogs学籍番号.keras')
```

In [11]:
# モデルの保存
model.save('catsdogs2220042.keras')

また、予測と犬猫の対応（train_data_it.class_indices）を、joblib.dumpで保存しておく。

```
# 犬猫の対応を保存
import joblib
joblib.dump(train_data_it.class_indices, 'animal.joblib')
```

In [12]:
# 犬猫の対応を保存
import joblib
joblib.dump(train_data_it.class_indices, 'animal.joblib')

['animal.joblib']

### 2-5. モデル読み込みの確認
保存したモデルを、tensorflow.keras.models.load_modelを使用して読み込み、予測ができるか確認してみよう。

まず、保存したモデルを読み込む。
```
from tensorflow.keras.models import load_model
m = load_model('ファイルパス')
```

In [13]:
from tensorflow.keras.models import load_model
m = load_model('/content/catsdogs2220042.keras')

予測に使用する画像を読み込んで準備する。

※ ここではtf.keras.preprocessing.image.load_imgを使用してよい。

```
#画像の読み込み
img = tf.keras.preprocessing.image.load_img(
    'ファイルパス', target_size=(224, 224)
)
```
```
#画像を配列に変換
img_array = tf.keras.preprocessing.image.img_to_array(img)
# 軸の追加（1枚の画像のため）
img_array = img_array[tf.newaxis, ...]
# スケーリング（0～255 → 0～1）
img_array = img_array/255.0
```
```
# 予測
model.predict(img_array)
```

In [14]:
#画像の読み込みと確認
img = tf.keras.preprocessing.image.load_img(
    '/content/FELV-cat.jpg', target_size=(224, 224)
)

In [15]:
# 画像を配列に変換
img_array = tf.keras.preprocessing.image.img_to_array(img)
# 軸の追加（1枚の画像のため）
img_array = img_array[tf.newaxis, ...]
# スケーリング（0～255 → 0～1）
img_array = img_array/255.0

In [19]:
# 予測
pred1 = m.predict(img_array).round().astype(int)
list(train_data_it.class_indices.keys())[pred1[0][0]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


'cats'

## 3. モデルのダウンロード
←保存したファイル2つを、左からダウンロードしておく。

## 3. サーバ構築のための準備
開発環境とサーバを合わせるために、tensorflow、joblibのバージョンを確認しておく。
```
import tensorflow as tf
tf.__version__
```
```
!pip list | grep joblib
```



In [17]:
import tensorflow as tf
tf.__version__

'2.17.1'

In [18]:
!pip list | grep joblib

joblib                             1.4.2


# 提出について
このファイルに授業中のチェック項目はありません。実行し、クラス番号氏名の変更を確認出来たら、ダウンロードして提出してください。